In [2]:
import sys
import ff

# sys.path.insert(0, "/Users/raymondiacobacci/Downloads/rcwaControl/")
# import rcwa  # type: ignore

import matplotlib.pyplot as plt
import numpy as np

import pandas as pd

h = 6.626070e-34  # Js Planck's constant
c = 2.997925e8  # m/s speed of light
k_B = 1.380649e-23  # J/K Boltzmann constant
q = 1.602176e-19  # C elementary charge
l_i = np.arange(350, 3001, 1)  # nm wavelength range
T_e = 2073.15  # K emitter temperature

blackbody_radiation = (
    lambda lambda_i, T: (2 * h * c**2)
    / ((np.exp((h * c) / (k_B * T * lambda_i * 1e-9)) - 1) * lambda_i**5)
    * 1e32
)

B_i = blackbody_radiation(l_i, T_e)  # 2073.15K blackbody

with open("homogeneous-film-emissivity.txt", "r") as f:
    p = f.readlines()
efficiency_array = ff.parse_data(p)

blackbody_transmission_array = [i * B_i for i in efficiency_array]

n_all = pd.read_excel("/Users/raymondiacobacci/LeiteLab-6_8/n-allHTMats-2.xlsx")
k_all = pd.read_excel("/Users/raymondiacobacci/LeiteLab-6_8/k-allHTMats.xlsx")

nb_B = (
    lambda lambda_i, T: (2 * c)
    / ((np.exp((h * c) / (k_B * T * lambda_i * 1e-9)) - 1) * lambda_i**4)
    * 1e23
)
nb_B_e = nb_B(l_i, T_e)
T_PV = 300
nb_B_PV = nb_B(l_i, T_PV)
nk_AlN = n_all["AlN"] + 1j * k_all["AlN"]
nk_W = n_all["W"] + 1j * k_all["W"]

# Testing of Kim's TORCWA

In [53]:
'''
This tests the performance of the two situations
'''
# from torcwa import rcwa

emissivity_at_wavelength_declan = np.load("control_density.npy")

def plot_density(array):
    print(emissivity_at_wavelength_declan)
    print(array * B_i)
    print(emissivity_at_wavelength_declan - array * B_i)
    
    plt.plot(l_i, array * B_i, label="+ Rand Grating")
    plt.plot(l_i, emissivity_at_wavelength_declan, label="Control (Declan)")
    plt.plot(l_i, B_i, label="Blackbody radiation")
    plt.plot(l_i, emissivity_at_wavelength_declan - array * B_i, label="Difference in values")
    plt.axvline(
        x=np.argmax(B_i) + l_i[0], color="r", linestyle="--", label="Peak intensity"
    )
    plt.legend(loc="upper right")
    plt.ylabel("Power Density (W/cm^2/nm)")
    plt.xlabel("Wavelength (nm)")
    plt.title("Spectral Density")
    plt.show()

er = np.ones(shape = (100,))
# er[50:60] = 0
ur = np.ones(shape=er.shape)
grating_depth = .473
grating_thickness = 1
latticeVectors = [grating_thickness, 1]

emissivity_at_wavelength = []

air_layer = rcwa.Layer(n = 1, thickness = 9999)
for wavelength_index in range(2651): # We have 2651 points
    # wavelength_index = 1150
    AlN_grating = rcwa.Crystal(latticeVectors, er=(nk_AlN[wavelength_index]**2-1) * er + 1, ur=ur)
    AlN_layer = rcwa.Layer(crystal = AlN_grating, thickness = grating_depth)
    W_layer = rcwa.Layer(n=nk_W[wavelength_index], thickness=9998)
    stack = rcwa.LayerStack(
        AlN_layer, incident_layer=air_layer, transmission_layer=W_layer
    )
    source = rcwa.Source(wavelength=l_i[wavelength_index] / 1000.0)
    solver = rcwa.Solver(stack, source, 3)
    results = solver.solve()
    emissivity_at_wavelength.append(1 - results['RTot'])
    # break

print(emissivity_at_wavelength)
# plot_density(emissivity_at_wavelength)

NameError: name 'rcwa' is not defined

# Testing of Raymond's rcwa-ctrl

In [ ]:
import sys
import ff

# sys.path.insert(0, "/Users/raymondiacobacci/Downloads/rcwaControl/")
# import rcwa  # type: ignore

import matplotlib.pyplot as plt
import numpy as np

import pandas as pd

h = 6.626070e-34  # Js Planck's constant
c = 2.997925e8  # m/s speed of light
k_B = 1.380649e-23  # J/K Boltzmann constant
q = 1.602176e-19  # C elementary charge
l_i = np.arange(350, 3001, 1)  # nm wavelength range
T_e = 2073.15  # K emitter temperature

blackbody_radiation = (
    lambda lambda_i, T: (2 * h * c**2)
    / ((np.exp((h * c) / (k_B * T * lambda_i * 1e-9)) - 1) * lambda_i**5)
    * 1e32
)

B_i = blackbody_radiation(l_i, T_e)  # 2073.15K blackbody

with open("homogeneous-film-emissivity.txt", "r") as f:
    p = f.readlines()
efficiency_array = ff.parse_data(p)

blackbody_transmission_array = [i * B_i for i in efficiency_array]

n_all = pd.read_excel("/Users/raymondiacobacci/LeiteLab-6_8/n-allHTMats-2.xlsx")
k_all = pd.read_excel("/Users/raymondiacobacci/LeiteLab-6_8/k-allHTMats.xlsx")

nb_B = (
    lambda lambda_i, T: (2 * c)
    / ((np.exp((h * c) / (k_B * T * lambda_i * 1e-9)) - 1) * lambda_i**4)
    * 1e23
)
nb_B_e = nb_B(l_i, T_e)
T_PV = 300
nb_B_PV = nb_B(l_i, T_PV)
nk_AlN = n_all["AlN"] + 1j * k_all["AlN"]
nk_W = n_all["W"] + 1j * k_all["W"]

In [195]:
%load_ext autoreload
%autoreload 2

import numpy as np
import solver_v2 as rc
import pandas as pd

h = 6.626070e-34  # Js Planck's constant
c = 2.997925e8  # m/s speed of light
k_B = 1.380649e-23  # J/K Boltzmann constant
q = 1.602176e-19  # C elementary charge
l_i = np.arange(350, 3001, 1)  # nm wavelength range
T_e = 2073.15  # K emitter temperature

n_all = pd.read_excel("/Users/raymondiacobacci/LeiteLab-6_8/n-allHTMats-2.xlsx")
k_all = pd.read_excel("/Users/raymondiacobacci/LeiteLab-6_8/k-allHTMats.xlsx")

nb_B = (
    lambda lambda_i, T: (2 * c)
    / ((np.exp((h * c) / (k_B * T * lambda_i * 1e-9)) - 1) * lambda_i**4)
    * 1e23
)
nb_B_e = nb_B(l_i, T_e)
T_PV = 300
nb_B_PV = nb_B(l_i, T_PV)
nk_AlN = n_all["AlN"] + 1j * k_all["AlN"]
nk_W = n_all["W"] + 1j * k_all["W"]

er = np.array([x%2 for x in range(100)])
er = np.ones(shape = er.shape)
er[50:60] *= 2
ur = np.ones(shape = er.shape)

test_wavelength_index = 650

n_harmonics = 3

air_layer = rc.Layer(permittivity = er*0+1, permeability = ur, thickness = 9, n_harmonics = n_harmonics)
aln_grating = rc.Layer(
    # permittivity=1*er,
    permittivity=er*2,
    permeability=ur,
    thickness=.473,
    n_harmonics=n_harmonics,
)
w_layer = rc.Layer(permittivity = nk_W[test_wavelength_index] ** 2 * er, permeability = ur, thickness = 9, n_harmonics = n_harmonics)

w_layer = rc.Layer(permittivity = 1 ** 2 * er, permeability = ur, thickness = 9, n_harmonics = n_harmonics)

my_solver = rc.Solver(layer_stack = [air_layer, aln_grating, w_layer, air_layer], grating_period = 1, wavelength = 1, n_harmonics = n_harmonics)
# print(my_solver.soln())


# print(f'Emissivity: {my_solver.global_scattering_matrix[(2*n_harmonics + 1)*2:, :(2*n_harmonics + 1)*2]}')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Converged in 1 iterations.
Normal harmonics: 3, 10
12.122029371866812
0 (-0.18136279268146957+3.147505888918364e-17j)
1 (-0.18029750266027708+1.0084485972732263e-17j)
2 (-0.17939375593532286+5.740823890845477e-20j)
3 (-0.18900072459380243+4.027024116907333e-18j)
4 (-0.17939375593532694+2.6855116126978825e-18j)
5 (-0.18029750266026118+6.600932007726963e-18j)
6 (-0.18136279268148772+1.353048997412417e-17j)
7 (-0.18136279268148822-2.4858189113233116e-17j)
8 (-0.18029750266026523-1.1218127878951134e-17j)
9 (-0.17939375593532297+1.3222264132883213e-20j)
10 (-0.18900072459380252+1.382528200010278e-18j)
11 (-0.17939375593532114+5.668469475612723e-18j)
12 (-0.18029750266026473-6.867743136337867e-18j)
13 (-0.18136279268147032-1.1844908734404136e-17j)

Normal harmonics: 3, 10
1116.3477914200098
0 (5.31764496206162+0.9537070365593434j)
1 (-7.127735639437985-0.022933385088760475j)
2 (1.2556096748062249+0.202801

In [59]:
x = [[ 3.17412133e+02-1.42598736e-17j, -8.95850937e+00-2.60268757e+00j,
   1.19519934e+01+7.58496567e+00j, -1.16368588e+01-1.31994254e+01j,
   8.22457622e+00+1.74781141e+01j,  3.78479205e+00+7.53168143e-18j,
  -9.14745550e+00-2.65758149e+00j,  1.21100694e+01+7.68528371e+00j,
  -1.17505603e+01-1.33283944e+01j,  8.28919438e+00+1.76154347e+01j],
 [ 1.91947930e+00-5.57660287e-01j,  7.77913829e+01-4.66855970e-17j,
  -3.43208179e+00-9.97111933e-01j,  5.02727349e+00+3.19040477e+00j,
  -5.12145222e+00-5.80914725e+00j,  1.73053317e+00-5.02766361e-01j,
   1.03454765e+00-4.13939480e-17j, -3.62102792e+00-1.05200586e+00j,
   5.18534943e+00+3.29072281e+00j, -5.23515366e+00-5.93811621e+00j],
 [-1.58075935e-01+1.00318039e-01j,  1.88946130e-01-5.48939253e-02j,
  -2.20000000e+00+0.00000000e+00j,  1.88946130e-01+5.48939253e-02j,
  -1.58075935e-01-1.00318039e-01j,  0.00000000e+00+0.00000000e+00j,
   0.00000000e+00+0.00000000e+00j,  0.00000000e+00+0.00000000e+00j,
   0.00000000e+00+0.00000000e+00j,  0.00000000e+00+0.00000000e+00j],
 [-5.12145222e+00+5.80914725e+00j,  5.02727349e+00-3.19040477e+00j,
  -3.43208179e+00+9.97111933e-01j,  7.77913829e+01-7.49635717e-17j,
   1.91947930e+00+5.57660287e-01j, -5.23515366e+00+5.93811621e+00j,
   5.18534943e+00-3.29072281e+00j, -3.62102792e+00+1.05200586e+00j,
   1.03454765e+00-5.55111512e-17j,  1.73053317e+00+5.02766361e-01j],
 [ 8.22457622e+00-1.74781141e+01j, -1.16368588e+01+1.31994254e+01j,
   1.19519934e+01-7.58496567e+00j, -8.95850937e+00+2.60268757e+00j,
   3.17412133e+02+1.14246605e-16j,  8.28919438e+00-1.76154347e+01j,
  -1.17505603e+01+1.33283944e+01j,  1.21100694e+01-7.68528371e+00j,
  -9.14745550e+00+2.65758149e+00j,  3.78479205e+00+5.55111512e-17j],
 [ 3.78479205e+00-1.42598736e-17j, -9.14745550e+00-2.65758149e+00j,
   1.21100694e+01+7.68528371e+00j, -1.17505603e+01-1.33283944e+01j,
   8.28919438e+00+1.76154347e+01j,  3.17412133e+02+7.53168143e-18j,
  -8.95850937e+00-2.60268757e+00j,  1.19519934e+01+7.58496567e+00j,
  -1.16368588e+01-1.31994254e+01j,  8.22457622e+00+1.74781141e+01j],
 [ 1.73053317e+00-5.02766361e-01j,  1.03454765e+00+0.00000000e+00j,
  -3.62102792e+00-1.05200586e+00j,  5.18534943e+00+3.29072281e+00j,
  -5.23515366e+00-5.93811621e+00j,  1.91947930e+00-5.57660287e-01j,
   7.77913829e+01+0.00000000e+00j, -3.43208179e+00-9.97111933e-01j,
   5.02727349e+00+3.19040477e+00j, -5.12145222e+00-5.80914725e+00j],
 [ 0.00000000e+00+0.00000000e+00j,  0.00000000e+00+0.00000000e+00j,
   0.00000000e+00+0.00000000e+00j,  0.00000000e+00+0.00000000e+00j,
   0.00000000e+00+0.00000000e+00j, -1.58075935e-01+1.00318039e-01j,
   1.88946130e-01-5.48939253e-02j, -2.20000000e+00+0.00000000e+00j,
   1.88946130e-01+5.48939253e-02j, -1.58075935e-01-1.00318039e-01j],
 [-5.23515366e+00+5.93811621e+00j,  5.18534943e+00-3.29072281e+00j,
  -3.62102792e+00+1.05200586e+00j,  1.03454765e+00-5.55111512e-17j,
   1.73053317e+00+5.02766361e-01j, -5.12145222e+00+5.80914725e+00j,
   5.02727349e+00-3.19040477e+00j, -3.43208179e+00+9.97111933e-01j,
   7.77913829e+01-5.55111512e-17j,  1.91947930e+00+5.57660287e-01j],
 [ 8.28919438e+00-1.76154347e+01j, -1.17505603e+01+1.33283944e+01j,
   1.21100694e+01-7.68528371e+00j, -9.14745550e+00+2.65758149e+00j,
   3.78479205e+00+5.55111512e-17j,  8.22457622e+00-1.74781141e+01j,
  -1.16368588e+01+1.31994254e+01j,  1.19519934e+01-7.58496567e+00j,
  -8.95850937e+00+2.60268757e+00j,  3.17412133e+02+5.55111512e-17j]]

x = np.array(x)

# x += np.eye(x.shape[0])
print(np.linalg.cond(x))

vals, vecs = np.linalg.eig(x)

print(np.sum(np.abs(vecs @ vals @ np.linalg.inv(vecs) - x)))
print(np.sum(np.abs((x - np.diag(vals)) @ vecs)))


172.80580919376223
16656.073235236472
3500.4865903570667


In [ ]:
print(eigenvalues_qr)
print(vals)

[ -2.18501663-8.59121273e-15j  -2.16302393+1.64621955e-14j
  65.25071637+4.53917366e-15j  76.57283747-7.88441490e-13j
  76.94124251+7.75159471e-13j  88.53117753-2.52457125e-15j
 362.31888983-8.21222222e-15j 283.89176081+5.94096817e-16j
 313.47637162+1.94984117e-15j 313.77910802-4.22967332e-16j]
[-1.62722364e+18+1.75540198e+07j -4.65802354e-01-1.75947997e-10j
 -4.54296154e-01+1.66756171e-10j  1.65797621e-02-8.16199539e-12j
  1.41985421e-02-3.12711292e-03j  1.41985421e-02+3.12711287e-03j
  8.07449585e-03-4.01498127e-03j  8.07449486e-03+4.01498042e-03j
  2.10548891e-03+3.41045514e-03j  2.10548603e-03-3.41045423e-03j]


In [103]:
import numpy as np

def custom_eigendecomposition(X, max_iter=1000, tol=1e-12):
    """
    Compute the eigendecomposition of a complex, non-symmetric matrix X
    from scratch using a basic QR-iteration approach (without shifts).
    
    Returns
    -------
    V : ndarray of shape (n, n)
        The eigenvector matrix.
    D : ndarray of shape (n, n)
        The diagonal eigenvalue matrix (same order as used internally).
    res_decomp : ndarray of shape (n, n)
        The residual X - V @ D @ V^-1.
    res_evals : ndarray of shape (n, n)
        The residual X - D @ I, where I is the n x n identity.
        
    Notes
    -----
    1. This is a demonstration of the unshifted QR algorithm. It may converge
       slowly or fail to converge for certain matrices.
    2. We do not use np.linalg.eig or scipy.linalg.eig, but do use basic
       operations like np.linalg.inv for convenience.
    3. The order of the eigenvalues in D (and corresponding columns in V)
       is determined by the order in which the QR iteration produces
       the (quasi-)triangular form.
    4. For large or tricky matrices, consider more robust methods with shifts,
       deflation strategies, etc.
    """
    # Ensure X is a NumPy array of type complex (to avoid issues)
    A = np.array(X, dtype=np.complex128)
    n, m = A.shape
    if n != m:
        raise ValueError("Matrix must be square.")

    # 1) Convert A to (upper) Hessenberg form via Householder reflections.
    #    This step speeds up QR iteration and is standard. 
    H, Q_hess = _to_hessenberg(A)

    # 2) Perform iterative (unshifted) QR on the Hessenberg form.
    #    We accumulate the transformations (Q_k's) to eventually get eigenvectors.
    H_final, Q_total = _qr_iteration(H, max_iter=max_iter, tol=tol)

    # 3) At this point, H_final is (in principle) upper triangular if everything converged,
    #    and the diagonal entries of H_final are the eigenvalues in the internal order.
    #    The columns of V = Q_hess @ Q_total are the eigenvectors in that same order.
    V = Q_hess @ Q_total
    
    # 4) Extract the eigenvalues from the diagonal of H_final
    eigenvals = np.diag(H_final)
    
    # Build the diagonal eigenvalue matrix D, preserving the exact order:
    D = np.diag(eigenvals)

    # 5) Compute the residuals:
    #    res_decomp = X - V @ D @ V^-1
    #    res_evals  = X - D @ I
    V_inv = np.linalg.inv(V)
    res_decomp = A - V @ D @ V_inv
    
    # For X - D @ I, just do A - D:
    # Because D is n x n, I is n x n => D @ I = D
    I_n = np.eye(n, dtype=np.complex128)
    res_evals = A - D @ I_n

    return V, D, res_decomp, res_evals


def _to_hessenberg(A):
    """
    Reduce A to upper Hessenberg form using Householder transformations.
    Returns H, Q so that A = Q @ H @ Q^H   (Q^H is the conjugate transpose),
    where H is upper Hessenberg and Q is unitary.
    """
    A = A.copy()
    n = A.shape[0]
    Q_total = np.eye(n, dtype=np.complex128)

    for k in range(n-2):
        # 1) Construct the Householder vector to zero out subdiagonal entries below A[k, k].
        x = A[k+1:, k]
        # Norm of x
        norm_x = np.linalg.norm(x)
        if norm_x < 1e-15:
            continue
        
        # Householder vector v
        v = x.copy()
        # We'll flip the sign of x[0] to create the reflection
        v[0] += np.sign(x[0]) * norm_x if x[0] != 0 else norm_x
        # Normalize v
        v = v / np.linalg.norm(v)
        
        # 2) Apply the transformation to A from left and right (in practice, we only need left on A).
        # Build the reflection block: I - 2 v v^H
        # but note that v is (n-k-1) long, so we embed into an n x n matrix for the operation.
        H_k = np.eye(n, dtype=np.complex128)
        H_k_sub = np.eye(n-k-1, dtype=np.complex128) - 2.0 * np.outer(v, v.conjugate())
        H_k[k+1:, k+1:] = H_k_sub
        
        A = H_k @ A @ H_k.conjugate().T
        Q_total = Q_total @ H_k.conjugate().T

    return A, Q_total


def _qr_iteration(H, max_iter=1000, tol=1e-12):
    """
    Perform unshifted QR iteration on upper Hessenberg matrix H.
    Returns R, Qacc so that R ~ Qacc^H * H * Qacc, with R being (nearly) upper triangular.
    Qacc accumulates all Q_i from each QR factorization step, i.e.
        Qacc = Q_1 * Q_2 * ... * Q_m
    """
    n = H.shape[0]
    Qacc = np.eye(n, dtype=np.complex128)
    R = H.copy()

    for _ in range(max_iter):
        # Check if R is already (close to) upper triangular
        off_diag_norm = np.linalg.norm(np.tril(R, -1))
        if off_diag_norm < tol:
            break
        
        # Basic unshifted QR: factor R = Q * U
        Q, U = _qr_decomposition(R)
        R = U @ Q
        Qacc = Qacc @ Q

    return R, Qacc


def _qr_decomposition(M):
    """
    Compute the (complex) QR factorization of M via Householder reflections:
        M = Q * R
    Returns Q, R.
    """
    M = M.copy()
    n = M.shape[0]
    Q = np.eye(n, dtype=np.complex128)
    
    for k in range(n):
        # Create Householder vector to eliminate below diagonal in column k
        x = M[k:, k]
        norm_x = np.linalg.norm(x)
        if norm_x < 1e-15:
            continue
        
        v = x.copy()
        v[0] += np.sign(x[0]) * norm_x if x[0] != 0 else norm_x
        v = v / np.linalg.norm(v)
        
        # Apply reflection to M from the left
        H_k = np.eye(n, dtype=np.complex128)
        H_sub = np.eye(n-k, dtype=np.complex128) - 2.0 * np.outer(v, v.conjugate())
        H_k[k:, k:] = H_sub
        
        M = H_k @ M
        Q = Q @ H_k.conjugate().T

    return Q, M  # Q, R

v, d, res_decomp, res_evals = custom_eigendecomposition(x)
def generate_random_ordering(size):
    from random import shuffle
    """
    Generates a random ordering of integers from 0 to size-1.

    :param size: The upper limit (exclusive) for the range of integers.
    :return: A list containing the integers 0 through size-1 in random order.
    """
    numbers = list(range(size))  # Create a list of integers from 0 to size-1
    shuffle(numbers)     # Shuffle the list in place
    return numbers
for i in range(100):
    p = generate_random_ordering(v.shape[0])
    print(np.sum(np.abs((x - d[p]) @ v[p])))

6042.947765845896
6375.96398989381
6031.48239846127
5636.810694676065
5366.357420259065
6374.753599149117
5182.485507296641
5384.862002716356
5983.989398644345
5805.001513585571
4764.542491589923
5719.265666098274
5393.9700771412545
4907.641814249002
5123.028972650996
5044.691516646456
5753.070359988938
4757.046750809088
3996.9346500511606
5348.887735177451
5495.677492667537
5488.072607790199
5862.260778162257
4720.006510616253
6009.755068535583
6049.900608239732
5991.204987020221
4918.718103822171
4562.185920280385
5584.842853447081
5720.459652317824
5540.607040306283
5781.713110764399
5808.694261860809
5601.9274910960785
6003.481837883756
5764.29716152416
5228.716597958344
6093.551761331157
5516.182283250446
5820.800681875971
5857.674759541694
5785.845535366779
5646.284316728829
4943.804074107756
5919.682439759626
4929.899577977327
5273.1926706343365
5682.852343364825
5811.447591145012
5422.017720458001
5588.747549707209
5890.588025080123
5923.1172494797265
5757.663864807272
5719.058

In [80]:
import numpy as np
import scipy.linalg

def compute_eigendecomposition(X, check_tol=1e-8):
    """
    Computes the Eigendecomposition of a complex, non-symmetric matrix X using scipy.linalg.eig.
    Returns:
      - eigenvalues (np.ndarray): 1D array of eigenvalues
      - eigenvectors (np.ndarray): 2D array of eigenvectors as columns
      - Delta1 (np.ndarray): X - V @ Lambda @ V^-1
      - Delta2 (np.ndarray): X - Lambda @ I

    Parameters:
      X (np.ndarray): A complex, non-symmetric 2D matrix of shape (n,n).
      check_tol (float): Tolerance level used in printing whether results are 'small'.

    Notes:
      - For a correct decomposition, we want norm(Delta1) to be "close to zero",
        typically checked by, e.g., np.linalg.norm(Delta1) / np.linalg.norm(X) < some_tol.
      - The second difference, Delta2 = X - Lambda @ I, is less standard for verifying
        an eigendecomposition, but included per the user's request.
    """
    
    # Compute eigenvalues and eigenvectors using a robust method from SciPy
    w, V = scipy.linalg.eig(X)
    
    # Convert the array of eigenvalues w into a diagonal matrix Lambda
    Lambda = np.diag(w)
    
    # Compute V^-1
    V_inv = np.linalg.inv(V)
    
    # Compute Delta1 = X - V @ Lambda @ V^-1
    Delta1 = X - (V @ Lambda @ V_inv)
    
    # Create an identity matrix I of the same shape as X
    I = np.eye(X.shape[0], dtype=X.dtype)
    
    # Compute Delta2 = X - (Lambda @ I) = X - Lambda
    # Here, Lambda is already shape (n,n), so Lambda @ I == Lambda
    def generate_random_ordering(size):
            from random import shuffle
            """
            Generates a random ordering of integers from 0 to size-1.

            :param size: The upper limit (exclusive) for the range of integers.
            :return: A list containing the integers 0 through size-1 in random order.
            """
            numbers = list(range(size))  # Create a list of integers from 0 to size-1
            shuffle(numbers)     # Shuffle the list in place
            return numbers
    Delta2 = X - Lambda[:, generate_random_ordering(Lambda.shape[0])]
    
    # Optional: Check norms and report
    norm_X = np.linalg.norm(X)
    norm_D1 = np.linalg.norm(Delta1)
    norm_D2 = np.linalg.norm(Delta2)

    print("Eigenvalues:\n", w)
    print("\nEigenvectors (columns):\n", V)
    print(f"\n||Delta1|| = ||X - VΛV^-1|| = {norm_D1:.4e}")
    print(f"Relative error for Delta1: {norm_D1 / norm_X:.4e}")
    if norm_D1 / norm_X < check_tol:
        print("Delta1 is small (within tolerance).")
    else:
        print("Delta1 is relatively large, possibly due to ill-conditioning.")
    
    print(f"\n||Delta2|| = ||X - ΛI|| = {norm_D2:.4e}")
    print(f"Relative error for Delta2: {norm_D2 / norm_X:.4e}")
    if norm_D2 / norm_X < check_tol:
        print("Delta2 is small (within tolerance).")
    else:
        print("Delta2 is relatively large, but note that checking X ≈ ΛI is nonstandard.")
    
    return w, V, Delta1, Delta2

# --------------------- Example usage ---------------------
if __name__ == "__main__":
    # Create a random complex matrix with a high condition number (approx.)
    # One way to induce a high condition number is to make the singular values
    # vary greatly. We can do that using SVD manipulation.
    np.random.seed(0)
    U, _ , Vh = np.linalg.svd(np.random.randn(5,5) + 1j*np.random.randn(5,5))
    s = np.logspace(0, 2.23, 5)  # singular values that vary up to about 170
    # Construct a matrix with these singular values
    X_high_cond = (U * s) @ Vh
    
    print("Condition number of X_high_cond:",
          np.linalg.cond(X_high_cond))
    
    # Compute its eigendecomposition
    w, V, D1, D2 = compute_eigendecomposition(X_high_cond)


Condition number of X_high_cond: 169.8243652461735
Eigenvalues:
 [-0.83703646+109.59192243j 31.45789446  +5.04775602j
 -2.40932088 +18.23347299j  1.002928    -0.98986729j
  2.36663134  +3.4128291j ]

Eigenvectors (columns):
 [[ 0.26477772+0.046577j    0.16484854-0.16930743j -0.31316841-0.03337364j
   0.61500206+0.j         -0.15516467+0.3537188j ]
 [ 0.17973931+0.42296905j  0.63223375+0.j         -0.43997915+0.20520133j
   0.00479848-0.27109328j -0.34732835+0.28087514j]
 [ 0.67512721+0.j          0.33234328+0.46137788j  0.18766741-0.41846479j
   0.06172858-0.09842965j -0.48684881+0.05662079j]
 [ 0.01468208+0.43379971j -0.12980245+0.32463162j  0.50942768+0.j
   0.13028952-0.57824319j  0.57641769+0.j        ]
 [-0.13540838+0.23234537j -0.27778573+0.14737557j  0.31631113-0.30857961j
  -0.1853994 -0.38606421j -0.13040315-0.24857413j]]

||Delta1|| = ||X - VΛV^-1|| = 1.3660e-13
Relative error for Delta1: 7.7289e-16
Delta1 is small (within tolerance).

||Delta2|| = ||X - ΛI|| = 2.1690e+02
Rel

In [85]:
import numpy as np

def householder_hessenberg(A):
    """
    Transform a general square matrix A into upper Hessenberg form H = Q^H * A * Q
    using Householder reflectors. Returns H, Q.
    In complex arithmetic, we use Q^H (conjugate transpose).
    """
    A = A.copy()
    n = A.shape[0]
    Q_accum = np.eye(n, dtype=A.dtype)
    
    for k in range(n-2):
        # Extract the Householder vector from the k-th column below diagonal
        x = A[k+1:, k].copy()
        if np.allclose(x, 0):
            continue
        
        # Compute norm and form the reflection
        alpha = -np.sign(x[0].real) * np.linalg.norm(x)
        u = x.copy()
        u[0] -= alpha
        # Householder vector
        norm_u = np.linalg.norm(u)
        if norm_u < 1e-15:
            continue
        v = u / norm_u
        
        # Apply reflection to A on the left
        A[k+1:, k:] -= 2.0 * np.outer(v, np.dot(np.conj(v), A[k+1:, k:]))
        # Apply reflection to A on the right
        A[:, k+1:] -= 2.0 * np.outer(np.dot(A[:, k+1:], v), np.conj(v))
        
        # Accumulate the transformations into Q_accum
        Htemp = np.eye(n, dtype=A.dtype)
        Htemp_k = np.eye(n - (k+1), dtype=A.dtype) - 2.0 * np.outer(v, np.conj(v))
        Htemp[k+1:, k+1:] = Htemp_k
        Q_accum = Q_accum @ Htemp  # For complex, this accumulates from the left
    
    return A, Q_accum

def shifted_qr_iteration(H, Q_global, tol=1e-12, max_iters=1000):
    """
    Perform shifted QR iterations on the (sub)matrix H (already in Hessenberg).
    Q_global accumulates the transformations for eigenvectors.
    
    Returns (H_updated, Q_updated, converged).
    """
    n = H.shape[0]
    converged = False
    iters = 0
    
    while iters < max_iters:
        iters += 1
        
        if n == 1:
            converged = True
            break
        
        # Check subdiagonal element for deflation
        subdiag = H[n-1, n-2]
        if abs(subdiag) < tol * (abs(H[n-2, n-2]) + abs(H[n-1, n-1])):
            converged = True
            break
        
        # Single shift: use bottom-right element as the shift
        shift = H[n-1, n-1]
        
        # Shifted matrix
        H_shifted = H.copy()
        H_shifted[np.diag_indices_from(H_shifted)] -= shift
        
        # Compute QR factorization of H_shifted
        Q, R = np.linalg.qr(H_shifted)
        
        # Reapply the shift: H = R @ Q + shift*I
        H = R @ Q
        H[np.diag_indices_from(H)] += shift
        
        # Accumulate the transformations into Q_global
        Q_full = np.eye(Q_global.shape[0], dtype=Q_global.dtype)
        Q_full[:n, :n] = Q
        Q_global = Q_global @ Q_full
        
        # Check for approximate convergence
        offdiag_norm = abs(H[n-1, n-2])  # Absolute value of the subdiagonal element
        if offdiag_norm < tol:
            converged = True
            break
    
    return H, Q_global, converged

def naive_francis_qr_eig(A, tol=1e-12, max_iters=1000):
    """
    Naive eigen-decomposition (values + vectors) for a complex, non-symmetric matrix A,
    using:
      1) Householder reduction to Hessenberg form.
      2) Shifted QR iteration with deflation.
    
    Returns:
      eigenvalues (w), eigenvectors (V)
    """
    n = A.shape[0]
    # 1) Convert to upper Hessenberg
    H, Q_hess = householder_hessenberg(A)
    
    # Initialize Q_global as Q_hess
    Q_global = Q_hess.copy()
    
    # We'll iterate from the bottom-right corner deflating 1x1 blocks at a time
    m = n
    while m > 0:
        # Apply shifted QR on the top-left (m x m) submatrix of H
        H_sub = H[:m, :m]
        H_updated, Q_updated, converged = shifted_qr_iteration(H_sub, Q_global[:m, :m], tol=tol, max_iters=max_iters)
        H[:m, :m] = H_updated
        Q_global[:m, :m] = Q_updated[:m, :m]
        
        if m == 1:
            # Single entry => direct eigenvalue
            m -= 1
            continue
        
        # Check subdiagonal for deflation
        subdiag = H[m-1, m-2]
        if abs(subdiag) < tol * (abs(H[m-2, m-2]) + abs(H[m-1, m-1])):
            # Deflate the bottom eigenvalue
            H[m-1, m-2] = 0.0
            m -= 1
        else:
            # If not deflated, continue without reducing m
            if not converged:
                print(f"Warning: QR iteration did not converge for m={m} after {max_iters} iterations.")
                m -= 1  # Proceed to the next eigenvalue to avoid infinite loop
    
    # After iteration, H should be (nearly) upper triangular (Schur form).
    # The eigenvalues are the diagonal of H.
    w = np.diag(H).copy()
    
    # V should approximate the eigenvector matrix
    V = Q_global.copy()
    
    return w, V

def compute_eigendecomposition_custom(X, check_tol=1e-8, max_iters=1000):
    """
    Computes the Eigendecomposition of a complex, non-symmetric matrix X *without*
    using built-in eigenvalue routines.
    
    Returns:
      - eigenvalues (w)
      - eigenvectors (V)
      - Delta1 = X - V @ diag(w) @ V^-1
      - Delta2 = X - diag(w) @ I
    
    Parameters:
      X (np.ndarray): A complex, non-symmetric 2D matrix of shape (n,n).
      check_tol (float): Tolerance level used in printing whether results are 'small'.
      max_iters (int): Maximum number of iterations for the QR algorithm.
    """
    # 1) Attempt to compute eigenvalues and "eigenvector" matrix
    w, V = naive_francis_qr_eig(X, tol=check_tol, max_iters=max_iters)
    
    # Form the diagonal matrix of eigenvalues
    Lambda = np.diag(w)
    
    # Invert V
    # (In practice, ill-conditioning may wreak havoc here.)
    try:
        V_inv = np.linalg.inv(V)
    except np.linalg.LinAlgError:
        raise ValueError("Eigenvector matrix V is singular or very ill-conditioned.")
    
    # Compute Delta1 = X - V @ Lambda @ V^-1
    Delta1 = X - (V @ Lambda @ V_inv)
    
    # Compute Delta2 = X - Lambda (since Lambda@I == Lambda)
    # Must ensure Lambda is same shape as X
    Delta2 = X - Lambda
    
    # Optional: compute norms
    norm_X = np.linalg.norm(X, ord='fro')
    norm_D1 = np.linalg.norm(Delta1, ord='fro')
    norm_D2 = np.linalg.norm(Delta2, ord='fro')
    
    print("===== Custom Eigensolver Results =====")
    print("Eigenvalues (diagonal of final H):")
    print(w)
    print("\nEigenvectors (V) [columns]:")
    print(V)
    
    print(f"\n||Delta1|| = ||X - VΛV^-1|| = {norm_D1:.4e}")
    rel_err1 = norm_D1 / (norm_X + 1e-15)
    print(f"Relative error for Delta1: {rel_err1:.4e}")
    if rel_err1 < check_tol:
        print("Delta1 is small (within tolerance).")
    else:
        print("Delta1 is relatively large (possible ill-conditioning).")
    
    print(f"\n||Delta2|| = ||X - ΛI|| = {norm_D2:.4e}")
    rel_err2 = norm_D2 / (norm_X + 1e-15)
    print(f"Relative error for Delta2: {rel_err2:.4e}")
    if rel_err2 < check_tol:
        print("Delta2 is small (within tolerance).")
    else:
        print("Delta2 is relatively large (note this is a nonstandard check).")
    
    return w, V, Delta1, Delta2

# -------------------------- Example usage --------------------------
if __name__ == "__main__":
    # Parameters
    tol = 1e-10
    max_iters = 1000  # Reduced to ensure faster convergence
    
    # Construct a random complex matrix with a high condition number
    np.random.seed(42)
    n = 5  # Adjust n as needed; n=5 should be manageable
    # Create random complex matrix
    random_matrix = np.random.randn(n, n) + 1j * np.random.randn(n, n)
    # Perform SVD to control singular values and condition number
    U, _, Vh = np.linalg.svd(random_matrix)
    # Create singular values with a high condition number
    # For condition number ~170, set s_max / s_min = 170
    # Here, we'll use logarithmically spaced singular values
    s = np.logspace(0, np.log10(170), n)
    # Construct the high condition number matrix
    X_high_cond = (U * s) @ Vh
    
    print("Condition number of X_high_cond (approx.):", np.linalg.cond(X_high_cond))
    
    # Compute its eigendecomposition using the custom routine
    w, V, D1, D2 = compute_eigendecomposition_custom(x, check_tol=tol, max_iters=max_iters)
    
    # Optional: Perform residual checks for each eigenpair
    def residual_check(X, w, V, tol=1e-8):
        """
        For each eigenpair (w[i], V[:,i]), check if X @ V[:,i] ≈ w[i] * V[:,i].
        """
        print("\n===== Residual Checks =====")
        for i in range(len(w)):
            lhs = X @ V[:, i]
            rhs = w[i] * V[:, i]
            residual = np.linalg.norm(lhs - rhs)
            print(f"Residual for eigenvalue {i}: {residual:.4e}")
            if residual < tol:
                print(f"Eigenpair {i} is within tolerance.")
            else:
                print(f"Eigenpair {i} exceeds tolerance.")
    
    residual_check(x, w, V, tol=tol)

Condition number of X_high_cond (approx.): 170.00000000000043
===== Custom Eigensolver Results =====
Eigenvalues (diagonal of final H):
[362.1151264 +0.33213299j 313.66537922-0.01106806j
 298.33854713-0.99947384j 283.04537969+0.29284885j
  95.78464391+1.49241979j  78.02681782+3.39706868j
  78.74757879-1.05551493j  66.05530786+0.19173203j
  -1.01344927-1.3393721j    1.64873205-2.3007734j ]

Eigenvectors (V) [columns]:
[[-4.95208794e-01-6.34306102e-02j -6.55357233e-01+2.05231691e-01j
  -5.28402507e-02+2.11108405e-01j  2.63153044e-02-4.78124709e-01j
  -1.08914383e-02+1.75915938e-04j -1.34217235e-02+6.61467784e-03j
  -1.12487073e-02+2.05109693e-03j  2.68876858e-02-3.57432092e-02j
  -2.41657323e-04+7.43737970e-04j  1.62702216e-03-2.55591091e-05j]
 [ 1.85494458e-02-4.67183450e-03j -9.98817714e-03+2.97343279e-02j
  -8.01607335e-02-1.96244446e-01j -2.52321823e-02-9.38200073e-02j
  -7.69346775e-01-1.13246376e-01j -4.04846620e-02+2.12438389e-01j
   2.06754892e-01+2.03703542e-01j -1.66216563e-01+

In [92]:
def eigendecomposition(matrix):
    """
    Compute the eigendecomposition of a matrix by solving the characteristic equation.

    Parameters:
        matrix (np.ndarray): A square matrix (n x n).

    Returns:
        eigenvalues (np.ndarray): Array of eigenvalues.
        eigenvectors (np.ndarray): Matrix of eigenvectors (columns are eigenvectors).
    """
    # Ensure the input is a square matrix
    if matrix.shape[0] != matrix.shape[1]:
        raise ValueError("Input must be a square matrix.")

    # Compute the characteristic polynomial coefficients
    coeffs = np.poly(matrix)

    # Find the eigenvalues (roots of the characteristic polynomial)
    eigenvalues = np.roots(coeffs)

    # Compute the eigenvectors for each eigenvalue
    eigenvectors = []
    for eigenvalue in eigenvalues:
        # Solve (A - lambda*I)x = 0
        shifted_matrix = matrix - eigenvalue * np.eye(matrix.shape[0])
        print(f'Determinant: {np.linalg.det(shifted_matrix)}')
        _, _, vh = np.linalg.svd(shifted_matrix)
        null_space = vh[-1, :]
        eigenvectors.append(null_space / np.linalg.norm(null_space))  # Normalize eigenvector

    eigenvectors = np.array(eigenvectors).T  # Combine eigenvectors into a matrix

    return eigenvalues, eigenvectors
# print(np.linalg.eig(x))
eigenvalues, eigenvectors = eigendecomposition(x)
print((x - np.diag(eigenvalues)) @ eigenvectors[0])
# print(eigendecomposition(x))

Determinant: (-86764027700.53421-985118.8279785186j)
Determinant: (-27636946998.808777+7865101.466236467j)
Determinant: (-27848600833.661182+7906204.054637957j)
Determinant: (-12621540057.836105+4040285.273667003j)
Determinant: (-369872.0582306937-9241.739616904788j)
Determinant: (431315.4123707739-8238.084115623502j)
Determinant: (443188.9743717199-8266.436512847953j)
Determinant: (569847.2732569338-9392.42550003107j)
Determinant: (-1412.689580762381+43.26145454439451j)
Determinant: (-249.36761568721826+58.07927684116869j)
[  29.67075024+9.64418875e+00j -123.13566598-2.05463798e+00j
 -157.92566736-1.14702827e-01j  106.2408493 -4.01068547e+00j
   22.24088261+1.05432638e+01j    5.39244187+9.75490143e+00j
   -4.87840095-2.18026751e+00j    2.56435297+4.71985021e-03j
    3.06009587-4.10364319e+00j  -14.75518674+1.05949599e+01j]


In [96]:
import numpy as np

def householder_hessenberg(A):
    """
    Reduce a matrix A to upper Hessenberg form H using Householder reflections.
    Returns H and the accumulated orthogonal/unitary matrix Q such that H = Q^H * A * Q
    """
    A = A.copy()
    n = A.shape[0]
    Q_accum = np.eye(n, dtype=A.dtype)

    for k in range(n-2):
        x = A[k+1:, k]
        norm_x = np.linalg.norm(x)
        if norm_x == 0:
            continue

        # Compute Householder vector
        # For numerical stability, use sign of x[0] to set alpha
        # Handle complex numbers by considering the phase
        if x[0] != 0:
            alpha = -np.abs(x[0]) * norm_x
        else:
            alpha = -norm_x

        u = x.copy()
        u[0] -= alpha
        norm_u = np.linalg.norm(u)
        if norm_u == 0:
            continue
        v = u / norm_u

        # Apply Householder transformation to A: H = (I - 2 v v^H) A (I - 2 v v^H)
        # Left multiplication
        A[k+1:, k:] -= 2.0 * np.outer(v, np.dot(np.conj(v), A[k+1:, k:]))
        # Right multiplication
        A[:, k+1:] -= 2.0 * np.outer(np.dot(A[:, k+1:], v), np.conj(v))

        # Accumulate Q: Q = Q * (I - 2 v v^H)
        Q = np.eye(n, dtype=A.dtype)
        Q[k+1:, k+1:] -= 2.0 * np.outer(v, np.conj(v))
        Q_accum = np.dot(Q_accum, Q)

    return A, Q_accum

def shifted_qr_step(H, Q_global, tol=1e-12):
    """
    Perform one shifted QR step on the upper Hessenberg matrix H.
    Updates Q_global with the QR transformation.
    Returns the updated H, updated Q_global, and a flag indicating deflation.
    """
    n = H.shape[0]
    if n == 1:
        return H, Q_global, False  # Nothing to do

    # Use the bottom-right element as the shift
    shift = H[-1, -1]
    # Shifted matrix
    H_shifted = H - shift * np.eye(n, dtype=H.dtype)

    # QR factorization
    Q, R = np.linalg.qr(H_shifted)

    # Form the new H
    H_new = R @ Q + shift * np.eye(n, dtype=H.dtype)

    # Update the global Q
    Q_global = Q_global @ Q

    # Check for deflation: if the (n, n-1) element is small
    deflation = False
    if abs(H_new[-1, -2]) < tol * (abs(H_new[-2, -2]) + abs(H_new[-1, -1])):
        deflation = True
        H_new[-1, -2] = 0.0  # Explicitly set to zero

    return H_new, Q_global, deflation

def compute_eigendecomposition_custom(X, check_tol=1e-8, max_total_iters=1000):
    """
    Computes the Eigendecomposition of a complex, non-symmetric matrix X without using built-in eigenvalue routines.
    
    Returns:
        - eigenvalues (w)
        - eigenvectors (V)
        - Delta1 = X - V @ diag(w) @ V^-1
        - Delta2 = X - diag(w) @ I
    """
    n = X.shape[0]
    # Step 1: Reduce to Hessenberg form
    H, Q = householder_hessenberg(X)

    eigenvalues = []
    eigenvectors = Q.copy()

    m = n
    total_iters = 0
    while m > 0 and total_iters < max_total_iters:
        if m == 1:
            # Last eigenvalue
            eigenvalues.append(H[0, 0])
            m -= 1
            continue

        # Perform shifted QR step
        H, Q, deflation = shifted_qr_step(H[:m, :m], Q[:m, :m], tol=check_tol)
        total_iters += 1

        if deflation:
            # The last eigenvalue has converged
            eigenvalues.append(H[m-1, m-1])
            # Remove the last row and column
            H = H[:m-1, :m-1]
            Q = Q[:m-1, :m-1]
            m -= 1

    if total_iters >= max_total_iters:
        print("Warning: Maximum number of iterations reached without full convergence.")

    # After convergence, the accumulated Q contains the eigenvectors
    V = Q

    # Now, construct eigenvalues list as array
    w = np.array(eigenvalues[::-1])  # Reverse to maintain original order

    # Form the diagonal matrix of eigenvalues
    Lambda = np.diag(w)

    # Invert V
    try:
        V_inv = np.linalg.inv(V)
    except np.linalg.LinAlgError:
        raise ValueError("Eigenvector matrix V is singular or very ill-conditioned.")

    # Compute Delta1 = X - V @ Lambda @ V_inv
    # Delta1 = X - (V @ Lambda @ V_inv)

    # Compute Delta2 = X - Lambda (since Lambda @ I == Lambda)
    Delta2 = X - Lambda

    # Compute norms
    norm_X = np.linalg.norm(X, ord='fro')
    # norm_D1 = np.linalg.norm(Delta1, ord='fro')
    norm_D2 = np.linalg.norm(Delta2, ord='fro')

    print("===== Custom Eigensolver Results =====")
    print("Eigenvalues:")
    print(w)
    print("\nEigenvectors (V) [columns]:")
    print(V)

    # print(f"\n||Delta1|| = ||X - VΛV^-1|| = {norm_D1:.4e}")
    # rel_err1 = norm_D1 / (norm_X + 1e-15)
    # print(f"Relative error for Delta1: {rel_err1:.4e}")
    # if rel_err1 < check_tol:
    #     print("Delta1 is small (within tolerance).")
    # else:
    #     print("Delta1 is relatively large (possible ill-conditioning).")

    print(f"\n||Delta2|| = ||X - ΛI|| = {norm_D2:.4e}")
    rel_err2 = norm_D2 / (norm_X + 1e-15)
    print(f"Relative error for Delta2: {rel_err2:.4e}")
    if rel_err2 < check_tol:
        print("Delta2 is small (within tolerance).")
    else:
        print("Delta2 is relatively large (note this is a nonstandard check).")

    return w, V, Delta2

# Residual Checks
def residual_check(X, w, V, tol=1e-8):
    """
    For each eigenpair (w[i], V[:,i]), check if X @ V[:,i] ≈ w[i] * V[:,i].
    """
    print("\n===== Residual Checks =====")
    for i in range(len(w)):
        lhs = X @ V[:, i]
        rhs = w[i] * V[:, i]
        residual = np.linalg.norm(lhs - rhs)
        print(f"Residual for eigenvalue {i}: {residual:.4e}")
        if residual < tol:
            print(f"Eigenpair {i} is within tolerance.")
        else:
            print(f"Eigenpair {i} exceeds tolerance.")

# Determinant Residual Checks
def determinant_residuals(X, w, tol=1e-8):
    """
    For each eigenvalue λ_i, compute det(X - λ_i I) and check if it's close to zero.
    """
    n = X.shape[0]
    residuals = []
    print("\n===== Determinant Residual Checks =====")
    for i in range(len(w)):
        lambda_i = w[i]
        M_shift = X - lambda_i * np.eye(n, dtype=X.dtype)
        det_residual = np.abs(np.linalg.det(M_shift))
        residuals.append(det_residual)
        print(f"det(M - λ_{i}I) = {det_residual:.4e}")
        if det_residual < tol:
            print(f"Eigenvalue {i} satisfies det(M - λI) ≈ 0 within tolerance.")
        else:
            print(f"Eigenvalue {i} does NOT satisfy det(M - λI) ≈ 0 within tolerance.")
    return residuals

# -------------------------- Example usage --------------------------
if __name__ == "__main__":
    # Parameters
    tol = 1e-8
    max_total_iters = 1000  # Total maximum iterations

    # Construct a random complex matrix with a high condition number
    np.random.seed(42)
    n = 5  # Matrix size
    # Create random complex matrix
    random_matrix = np.random.randn(n, n) + 1j * np.random.randn(n, n)
    # Perform SVD to control singular values and condition number
    U, s, Vh = np.linalg.svd(random_matrix)
    # Create singular values with a high condition number
    # For condition number ~170, set s_max / s_min = 170
    # Here, we'll use logarithmically spaced singular values
    s = np.logspace(0, np.log10(170), n)
    # Construct the high condition number matrix
    X_high_cond = (U * s) @ Vh

    print("Condition number of X_high_cond (approx.):", np.linalg.cond(x))

    # Compute its eigendecomposition using the custom routine
    w, V, D2 = compute_eigendecomposition_custom(x, check_tol=tol, max_total_iters=max_total_iters)

    # Perform residual checks for each eigenpair
    residual_check(X_high_cond, w, V, tol=tol)

    # Perform determinant residual checks
    determinant_residuals(X_high_cond, w, tol=tol)


Condition number of X_high_cond (approx.): 172.80580919376223
===== Custom Eigensolver Results =====
Eigenvalues:
[336.8918491   +2.50262333j 303.40842267  +7.07458972j
 282.04700687  -6.41755199j 183.48820521-128.15796108j
 169.33910053+130.22904234j  -2.8179313   +0.53703807j
  66.64714281  -7.15966053j  73.13886274  +4.48385277j
  84.85196577  +3.4631045j   79.4194392   -6.55507713j]

Eigenvectors (V) [columns]:
[[-0.84092258-0.02354238j]]

||Delta2|| = ||X - ΛI|| = 6.1045e+02
Relative error for Delta2: 9.2445e-01
Delta2 is relatively large (note this is a nonstandard check).

===== Residual Checks =====


ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 1 is different from 5)

In [56]:
def householder_reflection_complex(a):
    """
    Compute the Householder matrix for a complex vector a.
    """
    v = a.copy()
    norm_a = np.linalg.norm(a)
    if norm_a == 0:
        return np.eye(len(a), dtype=complex)
    # Add the magnitude to the first component to avoid cancellation
    v[0] += np.exp(1j * np.angle(a[0])) * norm_a
    v = v / np.linalg.norm(v)
    H = np.eye(len(a), dtype=complex) - 2.0 * np.outer(v, np.conj(v))
    return H

def qr_decomposition_householder_complex(A):
    """
    Perform QR Decomposition using Householder reflections for complex matrices.
    Returns matrices Q and R such that A = Q @ R
    """
    A = A.copy().astype(complex)  # Ensure A is of complex type
    n = A.shape[0]
    Q = np.eye(n, dtype=complex)

    for i in range(n-1):
        # Extract the vector to reflect
        a = A[i:, i]
        H = householder_reflection_complex(a)
        H_i = np.eye(n, dtype=complex)
        H_i[i:, i:] = H
        A = H_i @ A
        Q = Q @ H_i.conj().T  # Accumulate the orthogonal transformations
    R = A
    return Q, R

def qr_algorithm_shifted_complex(M, max_iterations=1000, tol=1e-6):
    """
    Compute the eigenvalues and eigenvectors of matrix M using the QR Algorithm with shifts.
    Supports complex matrices.
    Returns a tuple (eigenvalues, eigenvectors)
    """
    A_k = M.copy().astype(complex)
    n = M.shape[0]
    Q_total = np.eye(n, dtype=complex)

    for iteration in range(max_iterations):
        # Shift: Use the bottom-right element as the shift
        mu = A_k[-1, -1]
        I = np.eye(n, dtype=complex)
        A_shifted = A_k - mu * I

        # Perform QR decomposition on the shifted matrix
        Q, R = qr_decomposition_householder_complex(A_shifted)

        # Update A_k
        A_k = R @ Q + mu * I

        # Accumulate the orthogonal transformations
        Q_total = Q_total @ Q

        # Check for convergence (A_k is nearly upper triangular)
        # We consider convergence if the lower sub-diagonal elements are below tol
        lower_triangle = np.tril(A_k, k=-1)
        if np.all(np.abs(lower_triangle) < tol):
            print(f"Converged after {iteration+1} iterations.")
            break
    else:
        print("QR Algorithm did not converge within the maximum number of iterations")

    # Extract eigenvalues from the diagonal
    eigenvalues = np.diag(A_k)

    # Eigenvectors are accumulated in Q_total
    eigenvectors = Q_total

    return eigenvalues, eigenvectors

def verify_eigendecomposition_qr_complex(M, eigenvalues, eigenvectors, tol=1e-3):
    """
    Verify that M ≈ V @ diag(lambdas) @ V_inv within a tolerance using the QR Algorithm results.
    Supports complex matrices.
    """
    V = eigenvectors
    D = np.diag(eigenvalues)
    try:
        V_inv = np.linalg.inv(V)
    except np.linalg.LinAlgError:
        print("Eigenvector matrix is singular and cannot be inverted.")
        return False
    M_reconstructed = V @ D @ V_inv
    error = np.sum(np.abs(M_reconstructed - M))
    print(f"Reconstruction Error: {error}")
    return np.allclose(M, M_reconstructed, atol=tol)

# Step 1: Compute Eigenvalues and Eigenvectors using Enhanced QR Algorithm
eigenvalues_qr, eigenvectors_qr = qr_algorithm_shifted_complex(x)

print("Eigenvalues (QR Algorithm):")
print(eigenvalues_qr)

print(np.sum(np.abs(x - np.diag(eigenvalues_qr)) @ eigenvectors_qr))

# Step 2: Verify Eigendecomposition
if verify_eigendecomposition_qr_complex(x, eigenvalues_qr, eigenvectors_qr, tol=1e-3):
    print("\nEigendecomposition verification passed.")
else:
    print("\nEigendecomposition verification failed.")

# Display Eigenvectors
for idx in range(len(eigenvalues_qr)):
    print(f"Eigenvector {idx+1} corresponding to Eigenvalue {eigenvalues_qr[idx]}:\n{eigenvectors_qr[:, idx]}\n")

QR Algorithm did not converge within the maximum number of iterations
Eigenvalues (QR Algorithm):
[ -2.18501663-8.59121273e-15j  -2.16302393+1.64621955e-14j
  65.25071637+4.53917366e-15j  76.57283747-7.88441490e-13j
  76.94124251+7.75159471e-13j  88.53117753-2.52457125e-15j
 362.31888983-8.21222222e-15j 283.89176081+5.94096817e-16j
 313.47637162+1.94984117e-15j 313.77910802-4.22967332e-16j]
(-342.98885574894064-886.8453437854123j)
Reconstruction Error: 428.0461477994405

Eigendecomposition verification failed.
Eigenvector 1 corresponding to Eigenvalue (-2.1850166347558684-8.591212725735237e-15j):
[-1.27698978e-11+0.02570655j -7.07532915e-03-0.02435344j
 -5.22300366e-01-0.82301368j -1.90231250e-02-0.01677114j
  2.32599796e-02+0.01094532j  1.14954103e-10+0.02641286j
 -7.90212315e-03-0.02719929j  1.12706023e-01+0.17759628j
 -2.12460896e-02-0.01873095j  2.38990736e-02+0.01124605j]

Eigenvector 2 corresponding to Eigenvalue (-2.163023928166865+1.64621954745179e-14j):
[-0.03862192-1.13309748

In [50]:
def faddeev_leverrier(M):
    """
    Compute the coefficients of the characteristic polynomial of matrix M
    using the Faddeev-LeVerrier algorithm.

    p(λ) = λ^n + c_{n-1}λ^{n-1} + ... + c_1λ + c_0
    """
    n = M.shape[0]
    I = np.eye(n, dtype=complex)
    c = []
    B = I.copy()

    for k in range(1, n + 1):
        MK = M @ B
        trace_MK = np.trace(MK)
        c_k = -trace_MK / k
        c.append(c_k)
        B = MK + c_k * I
    return c

def characteristic_polynomial(M):
    """
    Returns the coefficients of the characteristic polynomial of M.
    """
    c = faddeev_leverrier(M)
    # The polynomial is p(λ) = λ^n + c_{n-1}λ^{n-1} + ... + c_1λ + c_0
    # For numpy's roots function, coefficients should be in descending order
    # of powers, starting with λ^n
    poly_coeffs = [1] + c[::-1]  # Reverse c to match numpy's order
    return poly_coeffs

def find_eigenvalues(M):
    """
    Find the eigenvalues of matrix M by computing the roots of its
    characteristic polynomial.
    """
    poly_coeffs = characteristic_polynomial(M)
    eigenvalues = np.roots(poly_coeffs)
    return eigenvalues

def verify_characteristic_equation(M, eigenvalues, tol=1e-4):
    """
    Verify that det(M - lambda*I) = 0 for each eigenvalue lambda.

    Returns a list of tuples containing (lambda, det(M - lambda*I), is_close)
    """
    n = M.shape[0]
    I = np.eye(n, dtype=complex)
    verification_results = []

    for idx, lambda_i in enumerate(eigenvalues):
        M_shifted = M - lambda_i * I
        det = np.linalg.det(M_shifted)
        is_close = np.isclose(det, 0, atol=tol)
        verification_results.append((lambda_i, det, is_close))
        print(f"Eigenvalue {idx+1}: {lambda_i}")
        if not is_close:
            print(f"det(M - lambda*I) = {det}, which is not close to 0.\n")

    return verification_results

def compute_eigenvector(M, lambda_i, tol=1e-4):
    """
    Compute the eigenvector corresponding to the eigenvalue lambda_i of matrix M
    by finding the null space of (M - lambda_i * I).

    Parameters:
    - M: The input matrix.
    - lambda_i: The eigenvalue.
    - tol: Tolerance for determining the null space.

    Returns:
    - eigenvector: A normalized eigenvector corresponding to lambda_i.
    """
    n = M.shape[0]
    I = np.eye(n, dtype=complex)
    M_shifted = M - lambda_i * I

    # Perform Singular Value Decomposition
    U, S, Vh = np.linalg.svd(M_shifted)

    # The singular vector corresponding to the smallest singular value
    # is the last row of Vh (V transposed)
    eigenvector = Vh[-1, :]

    # Normalize the eigenvector
    norm = np.linalg.norm(eigenvector)
    if norm < tol:
        print(f"Warning: Eigenvector for lambda {lambda_i} has near-zero norm.")
        return eigenvector
    return eigenvector / norm

vals = find_eigenvalues(x)
print(f'Eigenvalues:\n{vals}')

verification_results = verify_characteristic_equation(x, vals, tol = 1e-4)
passed = all(result[2] for result in verification_results)
if passed:
    print("\nAll eigenvalues satisfy det(M - lambda*I) ≈ 0 within the specified tolerance.")
else:
    print("\nSome eigenvalues do not satisfy det(M - lambda*I) ≈ 0 within the specified tolerance.")
    print("Please check the eigenvalue computations or consider numerical precision adjustments.")

print(np.sum(np.abs(vecs @ np.diag(vals) @ np.linalg.inv(vecs) - x)))

Eigenvalues:
[-1.62722364e+18+1.75540198e+07j -4.65802354e-01-1.75947997e-10j
 -4.54296154e-01+1.66756171e-10j  1.65797621e-02-8.16199539e-12j
  1.41985421e-02-3.12711292e-03j  1.41985421e-02+3.12711287e-03j
  8.07449585e-03-4.01498127e-03j  8.07449486e-03+4.01498042e-03j
  2.10548891e-03+3.41045514e-03j  2.10548603e-03-3.41045423e-03j]
Eigenvalue 1: (-1.6272236397427763e+18+17554019.763293482j)
det(M - lambda*I) = (1.301584742487843e+182-1.4041121168104809e+172j), which is not close to 0.

Eigenvalue 2: (-0.4658023539896813-1.7594799731737984e-10j)
det(M - lambda*I) = (1.0354926437306094e+18-201526242.57520342j), which is not close to 0.

Eigenvalue 3: (-0.4542961544819681+1.6675617096237144e-10j)
det(M - lambda*I) = (1.048708302626043e+18+192063003.31987247j), which is not close to 0.

Eigenvalue 4: (0.016579762085802505-8.161995385114196e-12j)
det(M - lambda*I) = (1.650352702667705e+18-11420015.830268128j), which is not close to 0.

Eigenvalue 5: (0.014198542094137487-0.003127112923

In [45]:
def householder_reflection(a):
    """
    Compute the Householder matrix for a vector a.
    """
    a = a.astype(float)  # Ensure a is of float type
    v = a.copy()
    norm_a = np.linalg.norm(a)
    if norm_a == 0:
        raise ValueError("Zero vector provided for Householder reflection.")
    v[0] += np.copysign(norm_a, a[0])
    v = v / np.linalg.norm(v)
    H = np.eye(a.shape[0]) - 2.0 * np.outer(v, v)
    return H

def qr_decomposition_householder(A):
    """
    Perform QR Decomposition using Householder reflections.
    Returns matrices Q and R such that A = Q @ R
    """
    A = A.copy().astype(float)  # Ensure A is of float type
    n = A.shape[0]
    Q = np.eye(n)
    
    for i in range(n-1):
        # Extract the vector to reflect
        a = A[i:, i]
        if np.allclose(a, 0):
            continue  # Skip if the column is already zero below the diagonal
        H = householder_reflection(a)
        H_i = np.eye(n)
        H_i[i:, i:] = H
        A = H_i @ A
        Q = Q @ H_i.T  # Accumulate the orthogonal transformations
    R = A
    return Q, R

def is_upper_triangular(A, tol=1e-8):
    """
    Check if a matrix is upper triangular within a tolerance.
    """
    return np.allclose(A, np.triu(A), atol=tol)

def qr_algorithm_shifted(A, max_iterations=1000, tol=1e-10):
    """
    Compute the eigenvalues and eigenvectors of matrix A using the QR Algorithm with shifts.
    Returns a tuple (eigenvalues, eigenvectors)
    """
    A_k = A.copy().astype(float)  # Ensure A_k is of float type
    n = A.shape[0]
    Q_total = np.eye(n)
    
    for iteration in range(max_iterations):
        # Shift: Use the bottom-right element as the shift
        mu = A_k[-1, -1]
        I = np.eye(n)
        A_shifted = A_k - mu * I
        
        # Perform QR decomposition on the shifted matrix
        Q, R = qr_decomposition_householder(A_shifted)
        
        # Update A_k
        A_k = R @ Q + mu * I
        
        # Accumulate the orthogonal transformations
        Q_total = Q_total @ Q
        
        # Check for convergence (A_k is nearly upper triangular)
        if is_upper_triangular(A_k, tol=tol):
            print(f"Converged after {iteration+1} iterations.")
            break
    else:
        print("QR Algorithm did not converge within the maximum number of iterations")
    
    eigenvalues = np.diag(A_k)
    eigenvectors = Q_total
    return eigenvalues, eigenvectors

def verify_eigendecomposition(M, eigenvalues, eigenvectors, tol=1e-6):
    """
    Verify that M = V @ diag(lambdas) @ V_inv within a tolerance.
    """
    V = eigenvectors
    D = np.diag(eigenvalues)
    try:
        V_inv = np.linalg.inv(V)
    except np.linalg.LinAlgError:
        print("Eigenvector matrix is singular and cannot be inverted.")
        return False
    M_reconstructed = V @ D @ V_inv
    return np.allclose(M, M_reconstructed, atol=tol)

def generate_random_ordering(size):
    from random import shuffle
    """
    Generates a random ordering of integers from 0 to size-1.

    :param size: The upper limit (exclusive) for the range of integers.
    :return: A list containing the integers 0 through size-1 in random order.
    """
    numbers = list(range(size))  # Create a list of integers from 0 to size-1
    shuffle(numbers)     # Shuffle the list in place
    return numbers

def compute_characteristic_polynomial(M, eigenvalues, tol=1e-6):
    """
    Verify that det(M - lambda*I) = 0 for each eigenvalue lambda.
    """
    n = M.shape[0]
    for idx, lambda_i in enumerate(eigenvalues[generate_random_ordering(len(eigenvalues))]):
        det = np.linalg.det(M - lambda_i * np.eye(n))
        if not np.isclose(det, 0, atol=tol):
            print(f"det(M - {lambda_i}*I) = {det}, which is not close to 0.")
            return False
    return True

vals, vecs = qr_algorithm_shifted(x)

print(np.sum(np.abs(vecs @ np.diag(vals) @ np.linalg.inv(vecs) - x)))
print(compute_characteristic_polynomial(x, vecs))

/var/folders/5n/xz5rf9sn7kdf2905qmv18yrr0000gn/T/ipykernel_23834/960651666.py:48: ComplexWarning: Casting complex values to real discards the imaginary part
  A_k = A.copy().astype(float)  # Ensure A_k is of float type


QR Algorithm did not converge within the maximum number of iterations
687.4523761217251
det(M - [-9.85012801e-01  1.44138606e-01 -4.27505231e-15  2.38609138e-03
 -2.61292234e-04 -6.07893426e-02 -7.26118972e-02 -4.23055836e-18
 -3.22473799e-17  4.99851249e-04]*I) = (1.6310957128734912e+18+109558.24095587176j), which is not close to 0.
False


In [158]:
import numpy as np

# Example matrix
M = np.array([[4, 1], [1, 3]])

# Eigendecomposition
eigvals, eigvecs = np.linalg.eig(M)

# Verify determinant for each eigenvalue
for i, eigval in enumerate(eigvals):
    det = np.linalg.det(M - eigval * np.eye(M.shape[0]))
    print(f"Determinant for eigenvalue {eigval:.4f}: {det:.4e}")


Determinant for eigenvalue 4.6180: 2.2204e-16
Determinant for eigenvalue 2.3820: 1.7964e-16


In [297]:
7.795684e+01*0.707107+ -5.844232e-14*(0.687761-0.164269j)

(55.12382726187996+9.60026146408e-15j)

In [298]:
7.795684e+01*(0.414224-0.126281j)+ -5.844232e-14*(0.901372+0.000000j)

(32.29159409215995-9.84446771204j)

In [274]:
np.abs(0.9429905358928607+0.33281954452297624j)

0.999999999999993

In [190]:
0.85355339+0.02512627

0.87867966

In [191]:
0.17157288**2

0.029437253151494404

In [65]:
0.94299054**2+0.33281954j**2

(0.77846231232368+0j)

In [58]:
x=[[78,0,0,0,0,0],[0,-1,0,0,0,0],[0,0,78,0.1,0,0],[0,0,0,78,0,0],[0,0,0,0,-1,0],[0,0,0,0,0,78]]
print(x)
print(np.linalg.eig(x)[1])
print('Eigen computed')
print(x@np.linalg.eig(x)[1][:,3])
print('Value computed')
print(np.linalg.eig(x)[0][3]*np.linalg.eig(x)[1][:,3])
x1=[[ 7.79568352e+01+0.j , 0.00000000e+00+0.j , 0.00000000e+00+0.j,
  -5.84423150e-14+0.j,  0.00000000e+00+0.j,  0.00000000e+00+0.j],
 [ 0.00000000e+00+0.j, -1.00000000e+00+0.j,  0.00000000e+00+0.j,
   0.00000000e+00+0.j,  0.00000000e+00+0.j,  0.00000000e+00+0.j],
 [ 0.00000000e+00+0.j,  0.00000000e+00+0.j,  7.79568352e+01+0.j,
   0.00000000e+00+0.j,  0.00000000e+00+0.j, -5.84423150e-14+0.j],
 [ 5.84423150e-14+0.j , 0.00000000e+00+0.j  ,0.00000000e+00+0.j
,   7.79568352e+01+0.j,  0.00000000e+00+0.j , 0.00000000e+00+0.j],
 [ 0.00000000e+00+0.j  ,0.00000000e+00+0.j  ,0.00000000e+00+0.j
 ,  0.00000000e+00+0.j, -1.00000000e+00+0.j,  0.00000000e+00+0.j],
 [ 0.00000000e+00+0.j ,0.00000000e+00+0.j ,5.84423150e-14+0.j
,0.00000000e+00+0.j ,0.00000000e+00+0.j ,7.79568352e+01+0.j]]
print(x1)
print(np.linalg.eig(x1)[1])
x2=[[ 7.79568352e+01+0.j , 0.00000000e+00+0.j , 0.00000000e+00+0.j,
  0+0.j,  0.00000000e+00+0.j,  0.00000000e+00+0.j],
 [ 0.00000000e+00+0.j, -1.00000000e+00+0.j,  0.00000000e+00+0.j,
   0.00000000e+00+0.j,  0.00000000e+00+0.j,  0.00000000e+00+0.j],
 [ 0.00000000e+00+0.j,  0.00000000e+00+0.j,  7.79568352e+01+0.j,
   0.00000000e+00+0.j,  0.00000000e+00+0.j, 0+0.j],
 [ 0+0.j , 0.00000000e+00+0.j  ,0.00000000e+00+0.j
,   7.79568352e+01+0.j,  0.00000000e+00+0.j , 0.00000000e+00+0.j],
 [ 0.00000000e+00+0.j  ,0.00000000e+00+0.j  ,0.00000000e+00+0.j
 ,  0.00000000e+00+0.j, -1.00000000e+00+0.j,  0.00000000e+00+0.j],
 [ 0.00000000e+00+0.j ,0.00000000e+00+0.j ,0+0.j
,0.00000000e+00+0.j ,0.00000000e+00+0.j ,7.79568352e+01+0.j]]
print(x2)
print(np.linalg.eig(x2)[1])
print(np.sum(np.array(x1) - np.array(x2)))

[[78, 0, 0, 0, 0, 0], [0, -1, 0, 0, 0, 0], [0, 0, 78, 0.1, 0, 0], [0, 0, 0, 78, 0, 0], [0, 0, 0, 0, -1, 0], [0, 0, 0, 0, 0, 78]]
[[ 1.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  1.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  1.00000000e+00 -1.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.73194792e-13
   0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   1.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  1.00000000e+00]]
Eigen computed
[ 0.00000000e+00  0.00000000e+00 -7.80000000e+01  1.35091938e-11
  0.00000000e+00  0.00000000e+00]
Value computed
[ 0.00000000e+00  0.00000000e+00 -7.80000000e+01  1.35091938e-11
  0.00000000e+00  0.00000000e+00]
[[(77.9568352+0j), 0j, 0j, (-5.84423

In [62]:
p=[[ 39.4784176+0.j,   0.       +0.j,   0.       +0.j, -38.4784176+0.j,
    0.       +0.j,   0.       +0.j],
 [  0.       +0.j,   0.       +0.j,   0.       +0.j,   0.       +0.j,
    1.       +0.j,   0.       +0.j],
 [  0.       +0.j,   0.       +0.j,  39.4784176+0.j,   0.       +0.j,
    0.       +0.j, -38.4784176+0.j],
 [ 38.4784176+0.j,   0.       +0.j,   0.       +0.j, -39.4784176+0.j,
    0.       +0.j,   0.       +0.j],
 [  0.       +0.j,  -1.       +0.j,   0.       +0.j,   0.       +0.j,
    0.       +0.j,   0.       +0.j],
 [  0.       +0.j,   0.       +0.j,  38.4784176+0.j,   0.       +0.j,
    0.       +0.j, -39.4784176+0.j]]
q=p=[[ 39.4784176+0.j,   0.       +0.j,   0.       +0.j, -38.4784176+0.j,
    0.       +0.j,   0.       +0.j],
 [  0.       +0.j,   0.       +0.j,   0.       +0.j,   0.       +0.j,
    1.       +0.j,   0.       +0.j],
 [  0.       +0.j,   0.       +0.j,  39.4784176+0.j,   0.       +0.j,
    0.       +0.j, -38.4784176+0.j],
 [ 38.4784176+0.j,   0.       +0.j,   0.       +0.j, -39.4784176+0.j,
    0.       +0.j,   0.       +0.j],
 [  0.       +0.j,  -1.       +0.j,   0.       +0.j,   0.       +0.j,
    0.       +0.j,   0.       +0.j],
 [  0.       +0.j,   0.       +0.j,  38.4784176+0.j,   0.       +0.j,
    0.       +0.j, -39.4784176+0.j]]
p,q=np.array(p),np.array(q)
print(p@q)
print(np.matmul(p,q))
print(manual_matmul(p,q))

[[ 7.79568352e+01+0.j  0.00000000e+00+0.j  0.00000000e+00+0.j
   7.64441833e-14+0.j  0.00000000e+00+0.j  0.00000000e+00+0.j]
 [ 0.00000000e+00+0.j -1.00000000e+00+0.j  0.00000000e+00+0.j
   0.00000000e+00+0.j  0.00000000e+00+0.j  0.00000000e+00+0.j]
 [ 0.00000000e+00+0.j  0.00000000e+00+0.j  7.79568352e+01+0.j
   0.00000000e+00+0.j  0.00000000e+00+0.j  7.64441833e-14+0.j]
 [-7.64441833e-14+0.j  0.00000000e+00+0.j  0.00000000e+00+0.j
   7.79568352e+01+0.j  0.00000000e+00+0.j  0.00000000e+00+0.j]
 [ 0.00000000e+00+0.j  0.00000000e+00+0.j  0.00000000e+00+0.j
   0.00000000e+00+0.j -1.00000000e+00+0.j  0.00000000e+00+0.j]
 [ 0.00000000e+00+0.j  0.00000000e+00+0.j -7.64441833e-14+0.j
   0.00000000e+00+0.j  0.00000000e+00+0.j  7.79568352e+01+0.j]]
[[ 7.79568352e+01+0.j  0.00000000e+00+0.j  0.00000000e+00+0.j
   7.64441833e-14+0.j  0.00000000e+00+0.j  0.00000000e+00+0.j]
 [ 0.00000000e+00+0.j -1.00000000e+00+0.j  0.00000000e+00+0.j
   0.00000000e+00+0.j  0.00000000e+00+0.j  0.00000000e+00+0.j]

In [61]:
def manual_matmul(A, B):
    """
    Manually performs matrix multiplication between two NumPy arrays A and B.

    Parameters:
    - A: NumPy array of shape (m, n)
    - B: NumPy array of shape (n, p)

    Returns:
    - result: NumPy array of shape (m, p) resulting from A x B
    """

    # Get the dimensions of the input matrices
    a_rows, a_cols = A.shape
    b_rows, b_cols = B.shape

    # Check if the matrices can be multiplied
    if a_cols != b_rows:
        raise ValueError("Incompatible dimensions for matrix multiplication.")

    # Initialize the result matrix with zeros
    result = np.zeros((a_rows, b_cols), dtype=complex)

    # Perform the matrix multiplication manually
    for i in range(a_rows):
        for j in range(b_cols):
            for k in range(a_cols):  # or range(b_rows)
                result[i, j] += A[i, k] * B[k, j]
    return result
